# Comment Toxicity Severity Rater
- group by the same comment and find out if that comment is more often rated as more/ less toxic 
- transform the input data to be more / less toxic 0 and 1 // train on these


In [ ]:
#!pip install alt-profanity-check
!pip install joblib
!pip install scikit-learn
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor


In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


In [2]:
# remove special characters using the regular expression library
# updated with . and = 
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\*)|(\=\=)|(\~) | (\=) | (\.\.\.) |(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile(" (<br\s/><br\s/?)| (\n\n) | (\n) |(\.) |(-)|(/)|(:). ")

#re.escape (* )

# re.sub() // natalie uses this more often

# regex has a method to escape the char properly 
re.escape


# gridsearchcv() // can be slow 
if have a few val, can use 

# use randomsearch() // possible range // faster 

In [3]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [4]:
original_data = pd.read_csv("./data/validation_data.csv")

data_to_test = pd.read_csv('./data/comments_to_score.csv')

In [5]:
pd.set_option('display.max_columns', None)

original_data.head()



,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [6]:
#original_data.iloc[0]
#original_data['more_toxic'][0]

In [7]:
cleaned_data = original_data
cleaned_data

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [8]:
clean_less_toxic = clean_tweets(original_data["less_toxic"])
cleaned_data['less_toxic'] = pd.DataFrame(clean_less_toxic)
#cleaned_data

In [9]:
clean_more_toxic = clean_tweets(original_data["more_toxic"])
cleaned_data['more_toxic'] = pd.DataFrame(clean_more_toxic)
cleaned_data

,worker,less_toxic,more_toxic
0,313,this article sucks woo woo wooooooo,what=wher is your sexy pic gone from your main...
1,188,and yes people should recognize that but they ...,daphne guinness top of the mornin my favourite...
2,82,western mediayup because every crime in the en...,atom you dont believe actual photos of masturb...
3,347,and you removed it you numbskull i dont care w...,you seem to have sand in your vagina.might wan...
4,539,smelly vagina bluerasberry why dont you be a m...,hey way to support nazis you racist
...,...,...,...
30103,461,im sorry im not an admin i will give you three...,get out my large penis
30104,527,im sorry im not an admin i will give you three...,get out my large penis
30105,352,wow...are you out of your mind how was my edit...,piss off you slant eyed gook
30106,311,wow...are you out of your mind how was my edit...,piss off you slant eyed gook


In [10]:
np.random.seed(0)
    
cleaned_data_shuffle_row = cleaned_data.sample(frac = 1, random_state = 0)


cleaned_data_shuffle_row

,worker,less_toxic,more_toxic
14331,9,its not so much about the english language its...,i hate drmies he is an idiot
17914,431,just read what mr george mcfly said here that ...,i fuckingg love tacosthere the fuckingg best b...
14390,338,a barnstar for you the original barnstar your...,fuck you fuck you motherfucker go die in a hole
24121,386,i am one who thinks that zappas dad was of ara...,jimbo wales is a gay little fucker
22424,397,a cup of coffee for you get fuckd with coffee,dude im not angry at you i just think youre a ...
...,...,...,...
13123,422,please stop your disruptive editing if you co...,youre a mathsci sockpuppet arent you
19648,293,dude you both are dumb in the heavier than hea...,mahmoud ahmadinejadi added the categoryanti se...
9845,347,then i guess i eat poop,racisttrying to whitewash the terrorists fine ...
10799,728,i thought you were the administrator can you f...,just because she has light skin doesnt mean an...


In [11]:
total_row = cleaned_data_shuffle_row.shape[0]
total_row

30108

In [12]:
train_data = cleaned_data_shuffle_row[:int(total_row*0.6)]
train_data = train_data.reset_index(drop = True)
train_data

,worker,less_toxic,more_toxic
0,9,its not so much about the english language its...,i hate drmies he is an idiot
1,431,just read what mr george mcfly said here that ...,i fuckingg love tacosthere the fuckingg best b...
2,338,a barnstar for you the original barnstar your...,fuck you fuck you motherfucker go die in a hole
3,386,i am one who thinks that zappas dad was of ara...,jimbo wales is a gay little fucker
4,397,a cup of coffee for you get fuckd with coffee,dude im not angry at you i just think youre a ...
...,...,...,...
18059,30,i cant imagine anything more depressing than d...,i dont give a fuck.. it will get reposted so y...
18060,141,give a reason when you accuse someone of vand...,bullshit=there was no violation you removed fa...
18061,118,additional evidence to above one of the collec...,because you are a fggt do i know you because y...
18062,526,omg i am obssessed with twilight and it is my ...,hey gwernol yeah youre a schmuck face thats wh...


In [13]:
val_data = cleaned_data_shuffle_row[int(total_row*0.6): int(total_row * 0.8)]
val_data = val_data.reset_index(drop = True)
val_data

,worker,less_toxic,more_toxic
0,283,an important message block me i can live with...,blockedyour recent behaviour has been utterly ...
1,652,gibnews the truth it is a lie which has been f...,fuck you fred fuck you you bitch ass punk u ca...
2,504,the germaine greer quote is highly appropriate...,i hate you i hate you.
3,567,if this is the sort of community that wants to...,please keep your two cents to yourselfi wasnt ...
4,261,this is bull this is y2k all over again this i...,the cherokee comment was a concern because jvm...
...,...,...,...
6017,655,racisma word describing characteristics of a p...,just for the record castro apologised for his ...
6018,472,you are not sorry one damned bit you have yet ...,so called fan sites on wikipedia include info...
6019,131,its all about termination not branches branch...,mgnbar what is a stupid act hi consensus mean...
6020,530,thats really shameless weaseling.,moot is a faggot he claims you are a sock pup...


In [14]:
test_data = cleaned_data_shuffle_row[int(total_row*0.8):]
test_data = test_data.reset_index(drop = True)
test_data

,worker,less_toxic,more_toxic
0,126,palestinian christians hi you have made doz...,re reply at jimbo youre right here about cond...
1,130,stop vandalizing wikipedia.,you have been blocked for vandalism to userpag...
2,728,age you might want to reconsider advertising y...,no not really i picked the name to wind up the...
3,398,the good thing is that you didnt call me mf lol,shitush shitush who are youyou are blatantly a...
4,2,the bottom line is that you are a biased contr...,was jesus gayif you say no how do you know wer...
...,...,...,...
6017,422,please stop your disruptive editing if you co...,youre a mathsci sockpuppet arent you
6018,293,dude you both are dumb in the heavier than hea...,mahmoud ahmadinejadi added the categoryanti se...
6019,347,then i guess i eat poop,racisttrying to whitewash the terrorists fine ...
6020,728,i thought you were the administrator can you f...,just because she has light skin doesnt mean an...


In [15]:
train_data_col_shuffle = pd.DataFrame()
indices_moreToxic = []
col_1 = []
col_0 = []
random.seed(0)

for i in range(train_data.shape[0]):
    idx = round(random.uniform(0, 1))
    #print('index of more toxic',idx)
    
    indices_moreToxic.append(idx)#index of where the more toxic comment is 
        #1 for right column, 0 for left column 
    
    if idx == 1: 
        col_1.append( train_data['more_toxic'][i])
        col_0.append( train_data['less_toxic'][i])
        
    else:
        col_0.append(train_data['more_toxic'][i])
        col_1.append( train_data['less_toxic'][i])


train_data_col_shuffle['col_0'] = pd.Series(col_0)
#trans = pd.DataFrame(comment, columns = ['comment']) // another method of setting to df col

train_data_col_shuffle['col_1'] = pd.Series(col_1)
train_data_col_shuffle['more_toxic_col_idx'] = pd.DataFrame(indices_moreToxic)

train_data_col_shuffle


,col_0,col_1,more_toxic_col_idx
0,its not so much about the english language its...,i hate drmies he is an idiot,1
1,just read what mr george mcfly said here that ...,i fuckingg love tacosthere the fuckingg best b...,1
2,fuck you fuck you motherfucker go die in a hole,a barnstar for you the original barnstar your...,0
3,jimbo wales is a gay little fucker,i am one who thinks that zappas dad was of ara...,0
4,a cup of coffee for you get fuckd with coffee,dude im not angry at you i just think youre a ...,1
...,...,...,...
18059,i cant imagine anything more depressing than d...,i dont give a fuck.. it will get reposted so y...,1
18060,give a reason when you accuse someone of vand...,bullshit=there was no violation you removed fa...,1
18061,additional evidence to above one of the collec...,because you are a fggt do i know you because y...,1
18062,hey gwernol yeah youre a schmuck face thats wh...,omg i am obssessed with twilight and it is my ...,0


In [16]:
#clean the input for predition data
cleaned_text = clean_tweets(data_to_test['text'])

data_to_test['text'] = pd.DataFrame(cleaned_text)
#data_to_test

In [17]:
# extract the labels from the train data
#y = trans.score.values
#X = trans.comment.values

from sklearn.feature_extraction.text import CountVectorizer

# initalize vectorizer: vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english') #stop_words removed: a, the

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(cleaned_data_shuffle_row.more_toxic.values) + 
               list(cleaned_data_shuffle_row.less_toxic.values) + list(data_to_test.text.values))
    

# transform documents to document-term matrix
col0_vec = vectorizer.transform(train_data_col_shuffle.col_0)
col1_vec = vectorizer.transform(train_data_col_shuffle.col_1)
comments_2_score = vectorizer.transform(data_to_test["text"].values)
'''
# combine col0 and col1
col0_1_vec = vectorizer.transform(train_data_col_shuffle.col_0 + ' ' + train_data_col_shuffle.col_1)
'''

# for testing on the original clean more and less toxic col (unshuffled)
more_tox_test_vec = vectorizer.transform(test_data['more_toxic'].values)
less_tox_test_vec = vectorizer.transform(test_data['less_toxic'].values)

#val set 
more_tox_val_vec = vectorizer.transform(val_data['more_toxic'].values)
less_tox_val_vec = vectorizer.transform(val_data['less_toxic'].values)



In [18]:
more_tox_val_vec

<6022x46974 sparse matrix of type '<class 'numpy.int64'>'
	with 145363 stored elements in Compressed Sparse Row format>

In [19]:
less_tox_test_vec

<6022x46974 sparse matrix of type '<class 'numpy.int64'>'
	with 156890 stored elements in Compressed Sparse Row format>

In [20]:
col0_vec

<18064x46974 sparse matrix of type '<class 'numpy.int64'>'
	with 445559 stored elements in Compressed Sparse Row format>

In [21]:
empty_vec = vectorizer.transform([' ']*6022) #used for testing // a baseline comment to be 
#biranked against
empty_vec

<6022x46974 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [23]:
from scipy.stats import rankdata
# make a function to score the prediction and compare if the "lesser" toxic rated is lower prob of rated toxicity
def comp_toxicity (model, less_tox_val_vec, more_tox_val_vec):
    '''
    model is the trained model to be evaluated against the less and more toxic comments
    objective is to see if more toxic comments have a higher score

    less_tox_val_vec and more_tox_val_vec are the respective column in dataframe that contains the comments vectorized
    '''
    result = res = np.zeros((1,2))




    for i in range(less_tox_val_vec.shape[0]):
        result = rankdata([model.predict_proba(less_tox_val_vec[i]).tolist()[0][1], 
                 model.predict_proba(more_tox_val_vec[i]).tolist()[0][1]], method = 'ordinal' )
        result -= 1
        #print(result)
        res += result
        #print(res[0])
        '''
        if i == 5:
            break
        '''
    print ('final result: ',res[0])
    return res[0]



In [22]:
import scipy.sparse as sp

combined_2 = sp.hstack([col0_vec, col1_vec], format='csr') #concatenate the 2 columns for train
combined_2

<18064x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 901644 stored elements in Compressed Sparse Row format>

In [24]:
less_tox_empty_test_vec = sp.hstack([ empty_vec, less_tox_test_vec], format='csr')
less_tox_empty_vec = sp.hstack([empty_vec, less_tox_val_vec], format='csr') 
# need empty_vec on the left side, comp_toxicity assumes more toxic on the right -- toxicity prob score
less_tox_empty_vec

<6022x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 160583 stored elements in Compressed Sparse Row format>

In [25]:
more_tox_empty_vec = sp.hstack([ empty_vec, more_tox_val_vec], format='csr')
more_tox_empty_test_vec = sp.hstack([ empty_vec, more_tox_test_vec], format='csr')
more_tox_empty_test_vec

<6022x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 144111 stored elements in Compressed Sparse Row format>

In [27]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)
#print(pd.DataFrame(clf.predict(more_tox_empty_vec)).value_counts()) #does predict more toxic against baseline
#print(pd.DataFrame(clf.predict(less_tox_empty_vec)).value_counts()) # does predict ' ' as more toxic than non-toxic

/Users/ericling/Documents/GitHub/DataSci Projects/KaggleToxicity/Kaggle_ToxicityRater/kaggle_kernel/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


final result:  [2143. 3879.]


array([2143., 3879.])

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, random_state=1)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

In [ ]:
from sklearn import neighbors
#choose neighbours after the cv 
clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec, more_tox_empty_vec)

In [ ]:
C = 1.0
clf = svm.SVC(kernel='linear', C=C, probability = True)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

clf = svm.SVC(kernel='rbf', C=C, probability = True)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

clf = svm.SVC(kernel='sigmoid', C=C, probability = True)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

clf = svm.SVC(kernel='poly', C=C, probability = True)
clf.fit(combined_2, train_data_col_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)




# cross validation on different models

In [28]:
from sklearn.model_selection import cross_val_score


In [29]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
cv_scores = cross_val_score(clf, combined_2, train_data_col_shuffle.more_toxic_col_idx, cv=10)
cv_scores.mean()

/Users/ericling/Documents/GitHub/DataSci Projects/KaggleToxicity/Kaggle_ToxicityRater/kaggle_kernel/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ericling/Documents/GitHub/DataSci Projects/KaggleToxicity/Kaggle_ToxicityRater/kaggle_kernel/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

0.6305367461716801

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=10, random_state=1)
cv_scores = cross_val_score(clf, combined_2, train_data_col_shuffle.more_toxic_col_idx,
                            cv=10)

print ('random forest: ',cv_scores.mean())

In [ ]:
#KNN choosing best neighbour number 

from sklearn import neighbors


for n in range(1, 15):
    clf = neighbors.KNeighborsClassifier(n_neighbors=n)
    cv_scores = cross_val_score(clf,combined_2, train_data_col_shuffle.more_toxic_col_idx
                                , cv=10)
    print (n, cv_scores.mean())
    
#k = 8 , 3 are first and sec place for best predictor respectively

In [ ]:
#KNN 

from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=8)
cv_scores = cross_val_score(clf, combined_2, train_data_col_shuffle.more_toxic_col_idx,
                            cv=10)

cv_scores.mean()

In [30]:
#Naive Bayes
#usually not the best performance, but fast to train
from sklearn import preprocessing

from sklearn.naive_bayes import MultinomialNB

#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.MaxAbsScaler() #don't really need scaling since the input is already binary 
training_inputs_minmax = scaler.fit_transform(combined_2)

clf = MultinomialNB()
cv_scores = cross_val_score(clf, combined_2, train_data_col_shuffle.more_toxic_col_idx, cv=10)

print ('minmax scaled Baye\'s: ',cv_scores.mean())

minmax scaled Baye's:  0.6335798215503754


In [ ]:
#SVM with kernel tuning

from sklearn import svm

C = 1.0
svc = svm.SVC(kernel='linear', C=C)
cv_scores = cross_val_score(svc,combined_2, train_data_col_shuffle.more_toxic_col_idx
                            , cv=10)
print ('linear kernel', cv_scores.mean())


svc = svm.SVC(kernel='rbf', C=C)
cv_scores = cross_val_score(svc, combined_2, train_data_col_shuffle.more_toxic_col_idx,
                            cv=10)
cv_scores.mean()
print ('rbf kernel', cv_scores.mean())


svc = svm.SVC(kernel='sigmoid', C=C)
cv_scores = cross_val_score(svc, combined_2, train_data_col_shuffle.more_toxic_col_idx
                            , cv=10)
cv_scores.mean()
print ('sigmoid kernel', cv_scores.mean())

svc = svm.SVC(kernel='poly', C=C)
cv_scores = cross_val_score(svc, combined_2, train_data_col_shuffle.more_toxic_col_idx
                            , cv=10)
cv_scores.mean()
print ('poly kernel', cv_scores.mean())

# try on colab: keras lstm, xgb 

Didn't try LSTM on biranked training, because the other models' accuracy didn't seem to improve with this biranked training of data transformation vs. the training done on rating aggregation.

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier( eta = 0.6 ,objective = 'multi:softmax', num_class = 2) #eta higher starts losing acc -- lower is slower
cv_scores = cross_val_score(clf, combined_2, train_data_col_shuffle.more_toxic_col_idx
                            , cv=10)
print (cv_scores.mean())

# Improvement
Transfer learning on BERT sentiment analysis -- change out the last layer to fine tune the range of ranking capabilities. Can try to tune those pre-trained transformer models to biranked or aggregated data provided in this competition. See if this method offers higher prediction accuracy.